# EXAMPLE SESSIONNODES
In this notebook, we create and manipulate SessionNodes from different inputs.

In [25]:
#IMPORT PACKAGES
from rdflib import Graph,URIRef,Literal, RDFS,XSD
import rdflib
import os.path, time
import importlib
import numpy as np
import open3d as o3d
import ifcopenshell
import cv2
from PIL import Image
from ifcopenshell.util.selector import Selector

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

from rdflib import RDF

In [44]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
%autoreload 2

## 1. INPUT

In [46]:
path= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"test","testfiles" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 


In [47]:

#sessionGraph (single)
sessionGraphPath=os.path.join(path,'sessionGraph.ttl')
sessionGraph=Graph().parse(sessionGraphPath)
sessionPath=os.path.join(path,'SESSION',"17dc31bc-17f2-11ed-bdae-c8f75043ce59.ply")  
sessionMesh= o3d.io.read_triangle_mesh(sessionPath)
subject=next(s for s in sessionGraph.subjects())

#resourceGraph (data)
resourceGraphPath=os.path.join(path,'resourceGraph.ttl')
resourceGraph=Graph().parse(resourceGraphPath)
linkedSubjects=[s for s in resourceGraph.subjects()]

#combinedGraph (combined)
combinedGraphPath=os.path.join(path,'combinedGraph.ttl')
combinedGraph=Graph().parse(combinedGraphPath)

#NODES
meshNode=MeshNode(path=os.path.join(path,'Mesh','Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095339.obj'),getResource=True)
imgNode=ImageNode(xmpPath=os.path.join(path,'IMG','IMG_2174.xmp'),getResource=True)
pcdNode=PointCloudNode(path=os.path.join(path,'PCD','week22 photogrammetry - Cloud.pcd'),getResource=True)
bimNode=BIMNode(graphPath=os.path.join(path,'bimGraph1.ttl'),subject='file:///Basic_Wall_211_WA_Ff1_Glued_brickwork_sandlime_150mm_1118860_0KysUSO6T3_gOJKtAiUE7d',getResource=True)
nodeList=[meshNode,imgNode,pcdNode,bimNode]
resources=[meshNode.resource,imgNode.resource,pcdNode.resource,bimNode.resource]

#RESOURCES
resourcePath=os.path.join(path,"resources")
if not os.path.exists(resourcePath):
    os.mkdir(resourcePath)
   

In [2]:
exif = rdflib.Namespace('http://www.w3.org/2003/12/exif/ns#')
geo=rdflib.Namespace('http://www.opengis.net/ont/geosparql#') #coordinate system information
gom=rdflib.Namespace('https://w3id.org/gom#') # geometry representations => this is from mathias
omg=rdflib.Namespace('https://w3id.org/omg#') # geometry relations
fog=rdflib.Namespace('https://w3id.org/fog#')
v4d=rdflib.Namespace('https://w3id.org/v4d/core#')
openlabel=rdflib.Namespace('https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#')
e57=rdflib.Namespace('http://libe57.org#')
xcr=rdflib.Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
ifc=rdflib.Namespace('http://ifcowl.openbimstandards.org/IFC2X3_Final#')


In [31]:
# path='D:\geomapi\tests\testfiles'
from pathlib import Path
import pye57
import laspy

#PATH
path= Path('D:\geomapi') / "tests" / "testfiles"  

#RESOURCES
resourceGraphPath=path /  'graphs' / 'parking_resource_graph.ttl'
resourceGraph=Graph().parse(str(resourceGraphPath))
print(f'loaded {resourceGraphPath}')

#POINTCLOUD
pcdPath=path / 'pcd'/"parking.pcd"
pcd=o3d.io.read_point_cloud(str(pcdPath))
print(f'loaded {pcd}')

e57Path1=path / 'pcd'/"lidar.e57"
xmlPath=path / 'pcd'/"lidar.xml"
e57_1 = pye57.E57( str(e57Path1))
e571=e57_1.read_scan_raw(1) 
print(f'loaded {e57Path1}')

e57Path2=path / 'pcd'/"parking.e57"
e57_2 = pye57.E57( str(e57Path2))
e572=e57_2.read_scan_raw(0) 
print(f'loaded {e57Path2}')

lasPath=path / 'pcd'/"parking.las"
las=laspy.read(str(lasPath))
print(f'loaded {lasPath}')

pcdGraphpath=path / 'graphs' /  'pcd_graph.ttl'
pcdGraph=Graph().parse(pcdGraphpath)
print(f'loaded {pcdGraphpath}')           


#MESH
meshPath=path / 'mesh'/ 'parking.obj'
mesh=o3d.io.read_triangle_mesh(str(meshPath))
print(f'loaded {mesh}') 

meshGraphPath=path / 'graphs' /  'mesh_graph.ttl'
meshGraph=Graph().parse(str(meshGraphPath))
print(f'loaded {meshGraphPath}')           


#IFC
ifcPath=path / 'ifc' / "parking.ifc"
bimNodes=tl.ifc_to_nodes_multiprocessing(str(ifcPath)) #! Note: this uses geomapi functionality
print(f'loaded {len(bimNodes)} bimNodes from ifc file')

ifcGraphPath=path /  'graphs' / 'parking_ifc_graph.ttl'
ifcGraph=Graph().parse(str(ifcGraphPath))
print(f'loaded {ifcGraphPath}')           

#IMG
csvPath=path / 'IMG' / 'parking.csv' #! we don't do anything with the csv
imgGraphPath=path /  'graphs' / 'road_img_graph.ttl'
imgGraph=Graph().parse(str(imgGraphPath))
print(f'loaded {imgGraphPath}')    

imageXmpPath1 = path / 'IMG' / 'DJI_0085.xmp'
imagePath1=path / 'IMG' / "DJI_0085.JPG" 
image1=cv2.imread(str(imagePath1))
imageCartesianTransform1= np.array([[-8.13902571e-02,  6.83059476e-01 ,-7.25813597e-01,  5.18276221e+01],
                                    [ 9.96648497e-01,  4.97790854e-02, -6.49139139e-02 , 6.10007435e+01],
                                    [-8.20972697e-03, -7.28664391e-01, -6.84821733e-01,  1.50408221e+01],
                                    [ 0.00000000e+00 , 0.00000000e+00, 0.00000000e+00 , 1.00000000e+00]])
print(f'loaded {imagePath1}')           

imageXmpPath2 = path / 'IMG' / 'IMG_8834.xmp'
imagePath2=path / 'IMG' / "IMG_8834.JPG" 
image2=cv2.imread(str(imagePath2))
imageCartesianTransform2= np.array([[ 4.12555151e-01,  4.12058430e-02 ,-9.10000179e-01, 6.68850552e+01],
                                    [ 9.10841440e-01, -4.52553581e-03,  4.12731621e-01 , 4.52551195e+01],
                                    [ 1.28887160e-02 ,-9.99140430e-01 ,-3.93990225e-02 , 5.45377093e+00],
                                    [ 0.00000000e+00 , 0.00000000e+00  ,0.00000000e+00 , 1.00000000e+00]])
print(f'loaded {imagePath2}')    


loaded D:\geomapi\tests\testfiles\graphs\parking_resource_graph.ttl
loaded PointCloud with 556485 points.
loaded D:\geomapi\tests\testfiles\pcd\lidar.e57
loaded D:\geomapi\tests\testfiles\pcd\parking.e57
loaded D:\geomapi\tests\testfiles\pcd\parking.las
loaded D:\geomapi\tests\testfiles\graphs\pcd_graph.ttl
loaded TriangleMesh with 62143 points and 49999 triangles.
loaded D:\geomapi\tests\testfiles\graphs\mesh_graph.ttl
loaded 2379 bimNodes from ifc file
loaded D:\geomapi\tests\testfiles\graphs\parking_ifc_graph.ttl
loaded D:\geomapi\tests\testfiles\graphs\road_img_graph.ttl
loaded D:\geomapi\tests\testfiles\IMG\DJI_0085.JPG
loaded D:\geomapi\tests\testfiles\IMG\IMG_8834.JPG


In [32]:
# imgNodes=[geomapi.imgNode(xmlPath=imageXmpPath1),imgNode(xmlPath=imageXmpPath1)]
imgNode1=ImageNode(xmpPath=imageXmpPath1,getResource=True)
imgNode2=ImageNode(xmpPath=imageXmpPath2,getResource=True)
meshNode=MeshNode(path=meshPath,getResource=True)
pcdNode=PointCloudNode(path=pcdPath,getResource=True)



# node=SessionNode(resources=[])

In [41]:
print(bimNodes[2].cartesianTransform)

[[  1.           0.           0.         -15.76490316]
 [  0.           1.           0.         100.99234288]
 [  0.           0.           1.           5.1       ]
 [  0.           0.           0.           1.        ]]


In [43]:
node= SessionNode(linkedNodes=bimNodes[0:3]+[imgNode1,imgNode2,meshNode,pcdNode])
node.to_graph(graphPath=r'D:\geomapi\tests\testfiles\graphs\road_session_graph.ttl',save=True)
# resources=node.get_linked_resources()

# print(node.linkedNodes)
# print(len(node.linkedSubjects))
# print(node.subject)
# print(node.path)
# print(node.name)
# print(node.resource)
# print(node.linkedSubjects)
# print(resources)

# node.add_linked_nodes(Node())
# print(node.linkedNodes)
# print(node.linkedSubjects)
# print(len(node.linkedSubjects))




<Graph identifier=N5a64c73a2d5e41e6a1765056ca2cf860 (<class 'rdflib.graph.Graph'>)>